# ----Pré processamento base de dados de credito bancario----

## Tratamento de valores inconsistentes

In [1]:
import pandas as pd
base = pd.read_csv('C:/Users/682481/OneDrive - Indra/Escritorio/Python/AnaliseDeDadosDeCreditoBancario/DataBase/credit_data.csv')
df = pd.DataFrame(base) #Convertendo para tabela
df

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


In [2]:
df.describe() # Resumo sobre o DataFrame

,i#clientid,income,age,loan,c#default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [3]:
df.loc[df['age'] < 0]  #Localizando clientes com idades negativas

,i#clientid,income,age,loan,c#default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [4]:
media = df['age'][df.age > 0].mean() #Verificando a média das idades com valores positivos
media

40.92770044906149

In [5]:
df.loc[df.age < 0, 'age'] = media #Substituindo os valores negativos por essa média de idades
df.loc[df['age'] < 0] #Localizando clientes com idades negativas

,i#clientid,income,age,loan,c#default


In [6]:
df.describe()

,i#clientid,income,age,loan,c#default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.927700,4444.369695,0.141500
std,577.494589,14326.327119,13.261825,3045.410024,0.348624
min,1.000000,20014.489470,18.055189,1.377630,0.000000
25%,500.750000,32796.459717,29.072097,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


## Tratamento de valores faltantes 

In [7]:
pd.isnull(df).sum() #Verificando quantos dados nulos existem no DataFrame

i#clientid    0
income        0
age           3
loan          0
c#default     0
dtype: int64

In [8]:
df.loc[pd.isnull(df['age'])] # Verificando quais linhas da coluna 'age' possuem dados nulos

,i#clientid,income,age,loan,c#default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [9]:
# Dividindo atributos previsores e classificadores
previsores = df.iloc[:, 1:4].values #Pegamos todas as linhas do DF e Pegamos as 3 ultimas colunas, desconsiderando a coluna ID e a coluna Default
pd.DataFrame(previsores)

,0,1,2
0,66155.925095,59.017015,8106.532131
1,34415.153966,48.117153,6564.745018
2,57317.170063,63.108049,8020.953296
3,42709.534201,45.751972,6103.642260
4,66952.688845,18.584336,8770.099235
...,...,...,...
1995,59221.044874,48.518179,1926.729397
1996,69516.127573,23.162104,3503.176156
1997,44311.449262,28.017167,5522.786693
1998,43756.056605,63.971796,1622.722598


In [10]:
classe = df.iloc[:, 4].values #Pegamos todas as linhas do DF e Pegamos a ultima coluna
pd.DataFrame(classe)

,0
0,0
1,0
2,0
3,0
4,1
...,...
1995,0
1996,0
1997,1
1998,0


In [11]:
#Tratando valores nulos
import numpy as np
from sklearn.impute import SimpleImputer

In [12]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(previsores[:,0:3]) #Encaixando a variavel "iputer a base de dados - com o metodo "fit()""
previsores[:,0:3] = imputer.transform(previsores[:,0:3]) #Atribuindo a transformação ao atributo previsores


In [13]:
pd.isnull(previsores).sum() #Verificando quantos dados nulos existem no Atributo

0

## Escalonamento de atributos

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)#Padronização

In [16]:
pd.DataFrame(previsores) #Atributos com a mesma escala

,0,1,2
0,1.453934,1.365380,1.202819
1,-0.762176,0.542659,0.696427
2,0.836821,1.674171,1.174711
3,-0.183070,0.364136,0.544980
4,1.509563,-1.686475,1.420765
...,...,...,...
1995,0.969747,0.572929,-0.826907
1996,1.688540,-1.340946,-0.309130
1997,-0.071226,-0.974486,0.354201
1998,-0.110003,1.739367,-0.926756


## Divisão de base de dados em treinamento e teste

In [17]:
# Utilizaremos 25% da base de dados para teste (test_size=0.25)

from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25, random_state=0)

In [18]:
pd.DataFrame(previsores_treinamento)

,0,1,2
0,-1.375446,0.506310,0.109809
1,1.458264,-1.648940,-1.215015
2,-0.793568,0.225311,-0.433702
3,-1.415844,0.350702,-0.353626
4,-0.592833,-0.321025,0.694192
...,...,...,...
1495,-1.105585,1.036952,-0.731577
1496,-0.997519,0.679936,0.007063
1497,0.217382,-0.147045,1.408725
1498,0.587162,0.664354,0.679481


In [19]:
pd.DataFrame(previsores_teste)

,0,1,2
0,1.593016,-1.354359,2.582627
1,0.997698,0.998065,0.844187
2,-0.424853,0.558125,-1.157853
3,-0.763567,-1.160842,-0.773369
4,-0.514269,-0.988721,-1.361509
...,...,...,...
495,0.401921,0.842218,1.528171
496,-0.410393,-0.643554,-1.316222
497,1.374457,-1.057464,-1.125648
498,-1.570877,-0.634883,-0.369817


In [20]:
pd.DataFrame(classe_treinamento)

,0
0,0
1,0
2,0
3,0
4,0
...,...
1495,0
1496,0
1497,0
1498,0


In [21]:
pd.DataFrame(classe_teste)

,0
0,1
1,0
2,0
3,0
4,0
...,...
495,0
496,0
497,0
498,1


# ----Aprendizagem bayesiana----

In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
classificador = GaussianNB()

In [25]:
classificador.fit(previsores_treinamento,classe_treinamento) #treinando o algritmo

GaussianNB()

In [26]:
previsoes = classificador.predict(previsores_teste) #testanto a previsao do alogritmo

In [28]:
pd.DataFrame(previsoes) #previsao com erros, correto seria igual a classe_teste

,0
0,1
1,0
2,0
3,0
4,0
...,...
495,0
496,0
497,0
498,0


## Identificando o percentual de acerto do agoritmo

In [32]:
#comparando previsao com erros, com o correto classe_teste para identificar o percentual de acerto do agoritmo
from sklearn.metrics import confusion_matrix,accuracy_score

In [33]:
precisao = accuracy_score(classe_teste,previsoes)

In [37]:
precisao*100 #percentual de acerto do algoritmo de 94%

93.8

In [38]:
matriz = confusion_matrix(classe_teste,previsoes) #matriz de confusão, para verificar qual classe o algoritmo está acertando e qual ele está errando mais

In [40]:
pd.DataFrame(matriz) # O algoritmo acerta mais da classe 0 e erra mais da classe 1

,0,1
0,428,8
1,23,41


In [41]:
# Analisando a matriz de confusão 
# linha classe 0, não pagou - linha classe 1, pagou
# linha classe 0 - 428 registros acertados e 8 errados
# linha classe 1 - 41 registros acertados e 23 errados
# Para contar quantos registros foram classificados corretamente, basta somar os registos acertados (428 + 41 = 469)
# Ou seja, dos 500 registros colocados para teste, 469 foram classificados corretamente - efetividade de acerto do algoritmo de 93.8%
